In [1]:
import random

# ==== RATING CALCULATION ====

def compute_rating(wins, losses, runs_scored, runs_allowed, on_base, slugging):
    games = wins + losses
    # Handle case where games is 0 to prevent ZeroDivisionError
    if games == 0:
        return 0.0
    win_pct = wins / games
    run_diff_per_game = (runs_scored - runs_allowed) / games
    run_diff_norm = (run_diff_per_game + 3) / 6  # Normalize to [0, 1]
    pythag_winloss = (runs_scored ** 1.83) / ((runs_scored ** 1.83) + (runs_allowed ** 1.83))
    # Use ** for exponentiation, not ^
    return (pythag_winloss * 0.35) + (win_pct * 0.1) + (run_diff_norm * 0.3) + (on_base * 0.1) + (slugging * 0.15)


# ==== GAME & SERIES SIMULATION ====

def game_win_prob(team_rating, opp_rating, home_field_advantage=0):
    # Handle case where both ratings are 0 to prevent ZeroDivisionError
    if (team_rating + home_field_advantage + opp_rating) == 0:
        return 0.5 # Default to 50/50 if no rating information
    return (team_rating + home_field_advantage) / (team_rating + home_field_advantage + opp_rating)

def simulate_game_100_times(team_rating, opp_rating, is_home, hfa=0.05):
    team_wins = 0
    opp_wins = 0
    for _ in range(100): # Simulate each game 100 times
        prob = game_win_prob(team_rating, opp_rating, hfa if is_home else 0)
        if random.random() < prob:
            team_wins += 1
        else:
            opp_wins += 1
    return team_wins > opp_wins # Return True if team wins more simulations, False otherwise

def get_home_games(round_name, home_team_is_A):
    formats = {
        "wildcard":       [True, True, True],
        "divisional":     [True, True, False, False, True],
        "lcs":            [True, True, False, False, False, True, True],
        "world_series":   [True, True, False, False, False, True, True],
    }
    base = formats[round_name]
    return base if home_team_is_A else [not g for g in base]

def simulate_series(team_A, team_B, round_name):
    series_format = {
        "wildcard":     (2, 3),
        "divisional":   (3, 5),
        "lcs":          (4, 7),
        "world_series": (4, 7)
    }[round_name]

    wins_needed, max_games = series_format

    # HOME FIELD LOGIC
    if round_name == "world_series":
        # For World Series, the home field advantage is determined by the team with the better regular season record.
        # If records are tied, then by seed.
        if team_A["wins"] != team_B["wins"]:
            home_team_is_A = team_A["wins"] > team_B["wins"]
        else:
            home_team_is_A = team_A["seed"] < team_B["seed"]
    else:
        home_team_is_A = team_A["seed"] < team_B["seed"]

    home_games = get_home_games(round_name, home_team_is_A)
    home_team_in_series = team_A if home_team_is_A else team_B
    away_team_in_series = team_B if home_team_is_A else team_A


    team_A_wins = 0
    team_B_wins = 0

    print(f"\n🎯 {round_name.upper()} SERIES: {team_A['name']} (Seed {team_A['seed']}) vs {team_B['name']} (Seed {team_B['seed']})")
    print(f"🏟 Home field advantage: {home_team_in_series['name']}")

    for i in range(max_games):
        # Ensure 'i' is within the bounds of 'home_games' list
        if i >= len(home_games):
            break

        is_home_for_A = home_games[i]
        
        # Determine which team is home for this specific game
        current_home_team = team_A if is_home_for_A else team_B

        winner_A = simulate_game_100_times(team_A['rating'], team_B['rating'], is_home_for_A)

        if winner_A:
            team_A_wins += 1
            print(f"Game {i+1} (Home: {current_home_team['name']}): {team_A['name']} win.")
        else:
            team_B_wins += 1
            print(f"Game {i+1} (Home: {current_home_team['name']}): {team_B['name']} win.")

        if team_A_wins == wins_needed:
            print(f"🏆 {team_A['name']} win the {round_name} series {team_A_wins}-{team_B_wins}")
            return team_A
        elif team_B_wins == wins_needed:
            print(f"🏆 {team_B['name']} win the {round_name} series {team_B_wins}-{team_A_wins}")
            return team_B


# ==== POSTSEASON BRACKET SIMULATION ====

def simulate_postseason(teams_AL, teams_NL):
    print("\n⚾ SIMULATING MLB POSTSEASON BRACKET\n")

    # Wild Card Round
    ALWC1 = simulate_series(teams_AL[3], teams_AL[6], "wildcard")
    ALWC2 = simulate_series(teams_AL[4], teams_AL[5], "wildcard")
    NLWC1 = simulate_series(teams_NL[3], teams_NL[6], "wildcard")
    NLWC2 = simulate_series(teams_NL[4], teams_NL[5], "wildcard")

    # Division Series
    ALDS1 = simulate_series(teams_AL[1], ALWC1, "divisional")
    ALDS2 = simulate_series(teams_AL[2], ALWC2, "divisional")
    NLDS1 = simulate_series(teams_NL[1], NLWC1, "divisional")
    NLDS2 = simulate_series(teams_NL[2], NLWC2, "divisional")

    # League Championship
    ALCS = simulate_series(ALDS1, ALDS2, "lcs")
    NLCS = simulate_series(NLDS1, NLDS2, "lcs")

    # World Series
    WS_winner = simulate_series(ALCS, NLCS, "world_series")
    print(f"\n🏁 WORLD SERIES CHAMPION: {WS_winner['name']} 🏆\n")


# ==== TEAM DATA ====

def create_team(name, wins, losses, runs_scored, runs_allowed, on_base, slugging, seed):
    return {
        "name": name,
        "wins": wins,
        "losses": losses,
        "runs_scored": runs_scored,
        "runs_allowed": runs_allowed,
        "on_base": on_base,
        "slugging": slugging,
        "seed": seed,
        "rating": compute_rating(wins, losses, runs_scored, runs_allowed, on_base, slugging)
    }

teams_AL = [
    None, # 0 index is unused, keeping for 1-based seed indexing
    create_team("Toronto Blue Jays", 78, 56, 664, 604, 0.338, 0.431, 1),
    create_team("Detroit Tigers", 78, 57, 650, 565, 0.318, 0.423, 2),
    create_team("Houston Astros", 74, 60, 569, 550, 0.318, 0.402, 3),
    create_team("Boston Red Sox", 75, 60, 668, 560, 0.323, 0.425, 4),
    create_team("New York Yankees", 74, 60, 702, 577, 0.332, 0.458, 5),
    create_team("Seattle Mariners", 72, 62, 612, 616, 0.317, 0.410, 6),
]

teams_NL = [
    None, # 0 index is unused, keeping for 1-based seed indexing
    create_team("Milwaukee Brewers", 83, 52, 688, 529, 0.332, 0.407, 1),
    create_team("Los Angeles Dodgers", 77, 57, 694, 586, 0.330, 0.441, 2),
    create_team("Philadelphia Phillies", 77, 57, 644, 545, 0.328, 0.426, 3),
    create_team("Chicago Cubs", 76, 58, 656, 543, 0.319, 0.426, 4),
    create_team("San Diego Padres", 75, 59, 564, 510, 0.321, 0.385, 5),
    create_team("New York Mets", 72, 62, 622, 561, 0.327, 0.423, 6),
]

simulate_postseason(teams_AL, teams_NL)


⚾ SIMULATING MLB POSTSEASON BRACKET


🎯 WILDCARD SERIES: Houston Astros (Seed 3) vs Seattle Mariners (Seed 6)
🏟 Home field advantage: Houston Astros
Game 1 (Home: Houston Astros): Seattle Mariners win.
Game 2 (Home: Houston Astros): Houston Astros win.
Game 3 (Home: Houston Astros): Seattle Mariners win.
🏆 Seattle Mariners win the wildcard series 2-1

🎯 WILDCARD SERIES: Boston Red Sox (Seed 4) vs New York Yankees (Seed 5)
🏟 Home field advantage: Boston Red Sox
Game 1 (Home: Boston Red Sox): Boston Red Sox win.
Game 2 (Home: Boston Red Sox): Boston Red Sox win.
🏆 Boston Red Sox win the wildcard series 2-0

🎯 WILDCARD SERIES: Philadelphia Phillies (Seed 3) vs New York Mets (Seed 6)
🏟 Home field advantage: Philadelphia Phillies
Game 1 (Home: Philadelphia Phillies): Philadelphia Phillies win.
Game 2 (Home: Philadelphia Phillies): Philadelphia Phillies win.
🏆 Philadelphia Phillies win the wildcard series 2-0

🎯 WILDCARD SERIES: Chicago Cubs (Seed 4) vs San Diego Padres (Seed 5)
🏟 Home fiel